In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_parquet(r"C:\Users\basde\Downloads\vsb-power-line-fault-detection\train.parquet", engine='pyarrow')

In [3]:
trainmeta = pd.read_csv(r"C:\Users\basde\Downloads\vsb-power-line-fault-detection\metadata_train.csv")

In [9]:
train = train.T

In [10]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,799990,799991,799992,799993,799994,799995,799996,799997,799998,799999
0,18,18,17,18,18,18,19,18,18,17,...,18,18,17,17,18,19,19,17,19,17
1,1,0,-1,1,0,0,1,0,0,0,...,1,0,0,0,0,2,1,0,1,0
2,-19,-19,-20,-19,-19,-20,-18,-19,-20,-19,...,-19,-20,-21,-18,-19,-18,-19,-19,-18,-19
3,-16,-17,-17,-16,-16,-15,-16,-17,-18,-17,...,-15,-15,-15,-15,-15,-15,-15,-15,-14,-14
4,-5,-6,-6,-5,-5,-4,-5,-7,-7,-7,...,-4,-4,-4,-5,-4,-4,-4,-4,-3,-4


In [16]:
print(f"Train index type: {train.index.dtype}")
print(f"trainmeta['signal_id'] type: {trainmeta['signal_id'].dtype}")

Train index type: int64
trainmeta['signal_id'] type: int64


In [29]:
def extract_features_from_signal_chunk(signal_chunk):
    """
    Calculates features for a given chunk of signal data.
    """
    # Use np.float32 to cut memory usage for floating point numbers in half
    signals = signal_chunk.astype(np.float32)
    
    features = pd.DataFrame(index=signal_chunk.index)
    
    features['mean'] = signals.mean(axis=1)
    features['std'] = signals.std(axis=1)
    features['min'] = signals.min(axis=1)
    features['max'] = signals.max(axis=1)
    features['skew'] = signals.skew(axis=1)
    features['kurtosis'] = signals.kurtosis(axis=1)
    
    fft_coeffs = np.fft.fft(signals, axis=1)
    fft_magnitudes = np.abs(fft_coeffs)
    for i in range(1, 11):
        features[f'fft_mag_{i}'] = fft_magnitudes[:, i]
        
    return features

In [34]:
# 2. Define chunking parameters
chunk_size = 10
num_rows = len(train)
features_list = []

print(f"\nStarting feature extraction from the separate train object...")
print(f"Processing in chunks of {chunk_size} rows...")

# 3. The chunking loop now iterates over the `train` object
for start in range(0, num_rows, chunk_size):
    end = min(start + chunk_size, num_rows)
    
    # Get a slice (chunk) of the signal data
    signal_chunk = train.iloc[start:end]
    
    # Process the chunk using our simplified function
    chunk_features = extract_features_from_signal_chunk(signal_chunk)
    
    features_list.append(chunk_features)
    print(f"Processed rows {start} to {end-1}")

# 4. Combine all the results
final_features = pd.concat(features_list)

print("\nFeature extraction complete.")
print("Shape of the final feature matrix:", final_features.shape)
print("\n--- Final Feature DataFrame (First 5 rows) ---")
print(final_features.head())

# Optional Next Step: You can now combine your original metadata with these new features
# if you need them together for a supervised model.
# X_train_full = pd.concat([train, final_features], axis=1)
# print("\n--- Combined metadata and features (sample) ---")
# print(X_train_full.head())


Starting feature extraction from the separate train object...
Processing in chunks of 10 rows...
Processed rows 0 to 9
Processed rows 10 to 19


MemoryError: Unable to allocate 122. MiB for an array with shape (10, 800000) and data type complex128